In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
gene_x_sample = pd.read_csv(PATH["gene_x_sample.tsv"], sep="\t", index_col=0)

gene_sets = ccal.read_gmts(SETTING["gene_set_file_paths"])

In [ ]:
gene_set_x_information = pd.DataFrame(index=gene_sets.index)

gene_set_x_information["Size"] = gene_sets.apply(
    lambda gene_set_genes: gene_set_genes.dropna().size, axis=1
)

gene_set_x_information["N"] = gene_sets.apply(
    lambda gene_set_genes: sum(
        gene_set_gene in gene_x_sample.index for gene_set_gene in gene_set_genes
    ),
    axis=1,
)

gene_set_x_information["Fraction"] = (
    gene_set_x_information["N"] / gene_set_x_information["Size"]
)

selected_gene_sets = ccal.select_series_indices(
    gene_set_x_information["N"],
    ">",
    threshold=SETTING["gene_set_minimum_n"],
    title=dict(text="Gene Sets Ranked by the Number of Genes in the Data"),
    xaxis=dict(title="Rank"),
    yaxis=dict(title="N"),
) & ccal.select_series_indices(
    gene_set_x_information["Fraction"],
    ">",
    threshold=SETTING["gene_set_minimum_fraction"],
    title=dict(text="Gene Sets Ranked by the Fraction of Genes in the Data"),
    xaxis=dict(title="Rank"),
    yaxis=dict(title="Fraction"),
)

print(selected_gene_sets.size)

gene_set_x_information.loc[selected_gene_sets, "Good"] = 1

gene_set_x_information["Good"].fillna(0, inplace=True)

gene_set_x_information.to_csv(PATH["gene_set_x_information.tsv"], sep="\t")

gene_set_x_information.sort_values("Good")

In [ ]:
if SETTING["gene_sets_to_peek"] is not None:

    gene_set_x_information.reindex(SETTING["gene_sets_to_peek"]).sort_values("Good")

In [ ]:
select = np.full(gene_set_x_information.index.size, False)

select |= gene_set_x_information.index.str.lower().str.contains("metabol")

gene_set_x_information.loc[select].sort_values("Good")